<a href="https://colab.research.google.com/github/kyou0612/Weeklyreport/blob/main/tenpai_lstm_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import**

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import codecs, copy
import time
import numpy as np

# **準備**

In [8]:
train_path = '/content/drive/MyDrive/experiment/learning_data/new/data_train_2013_MjT_tnsp.txt'
test_path = '/content/drive/MyDrive/experiment/learning_data/new/data_test_2015_MjT_tnsp.txt'


# **モデル**

In [9]:
class Network(nn.Module):
  def __init__(self):
      super(Network, self).__init__()
      self.lstm = nn.LSTM(input_size = 36,
                  hidden_size = 32,                    
                  batch_first=True)
      self.output_linear_layer = nn.Sequential(
          nn.Linear(32, 16),
          nn.ReLU(), 
          nn.Dropout(p=0.2),         
          nn.Linear(16, 1),
          nn.Sigmoid()
        
      )
      nn.init.xavier_normal_(self.lstm.weight_ih_l0)
      nn.init.orthogonal_(self.lstm.weight_hh_l0)

  def forward(self, inputs):
      h , _=self.lstm(inputs) 
      output = self.output_linear_layer(h[:,:])

      return output

# **Train**

In [19]:
class Train():
  def __init__(self):
    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using {} device'.format(self.device))
    self.net = Network().to(self.device)
    self.criterion = nn.BCELoss()
    self.optimizer = torch.optim.Adam(self.net.parameters(),lr=0.00001,amsgrad=True)



  
  def train_step(self,inputs,labels):
    
    inputs = torch.from_numpy(inputs).to(torch.float32)
    labels = torch.from_numpy(labels).to(torch.float32)
    
    inputs =torch.Tensor(inputs).to(self.device)
    labels =torch.Tensor(labels).to(self.device)

    inputs = torch.reshape(inputs, (-1, inputs.shape[1],inputs.shape[2]))
    # labels = torch.reshape(labels, (-1, inputs.shape[1]))
    
    self.net.train()
    
    preds =self.net(inputs)
    
    
    # for i in range(preds.shape[1]):
    loss =self.criterion(preds[0],labels[0])
    
    self.optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_value_(self.net.parameters(), clip_value=2.0)
    self.optimizer.step()
    return loss,preds
  
  def test_step(self,inputs,labels):
    inputs = torch.from_numpy(inputs).to(torch.float32)
    labels = torch.from_numpy(labels).to(torch.float32)
    
    inputs =torch.Tensor(inputs).to(self.device)
    labels =torch.Tensor(labels).to(self.device)
    inputs = torch.reshape(inputs, (-1, inputs.shape[1],inputs.shape[2]))
    self.net.eval()
    preds = self.net(inputs)    
    loss = self.criterion(preds[0], labels[0])
    res = torch.ge(preds[0],0.5).to(torch.float32)
    acc = torch.mean(torch.eq(res,labels[0]).to(torch.float32))

    
    return loss,acc, preds

  def train(self,train_inputs, train_labels, test_inputs, test_labels,epochs=25,batch_size=1, input_size=36):
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(epochs):
      print('-------------')
      print('Epoch {}/{}'.format(epoch+1, epochs))
      print('-------------')
      train_loss = 0.
      test_loss = 0.
      test_acc = 0
      test_tenpai = 0
      for i in range(len(train_inputs)):
        loss,_=self.train_step(train_inputs[i],train_labels[i])
        train_loss += loss.item()
        
      for i in range(len(test_inputs)):
        loss2,acc,_=self.test_step(test_inputs[i],test_labels[i])
        test_tenpai += np.mean(test_labels[i])        
        test_loss += loss2.item()
        test_acc += acc.item()
      train_loss /= float(len(train_inputs))
      test_loss /= float(len(test_inputs))
      test_acc /= float(len(test_inputs))
      test_tenpai /= float(len(test_inputs))
      print('loss: {:.3}, test_loss: {:.3},test_acc: {:.3},test_baseline: {:.3}'.format(train_loss, test_loss,test_acc,(1-test_tenpai)))




# **dateset**

In [11]:
class Dataset():
  def __init__(self):
    self.x_feature_vector =[[[0] for j in range(36)] for i in range(96)]   
    self.y = [[0] for i in range(4)]
    self.round = [0]*4 #東南西北    
    self.lastplayernum = 0

  def make_feature_vector_forone(self,line): 
    line = line.split()
    
    xmk_round=[-1]*4
    jikaze = int(line[10].split(':')[1])
    playernum = jikaze - 27

    lichi = int(line[3].split(':')[1])
    # self.x_feature_vector[self.round[playernum]+playernum*24][36] = [lichi] 
    sute = line[6].split(':')[1].split(',')
    tsumogiri = line[7].split(':')[1].split(',')
    red  = line[8].split(':')[1].split(',')[-1]
    dora = [int(s.split(',')[0]) for s in line[0].split(':')[1:]]
    dora_round = [int(s.split(',')[1]) for s in line[0].split(':')[1:]]
    bakaze = int(line[9].split(':')[1])
    
    xmk_count = -1
    if len(sute) == 1 and playernum == 0:
      self.x_feature_vector =[[[0] for j in range(36)] for i in range(96)]
      # y = [0]
      # lichi = [0]*4
      self.round = [0]*4 #東南西北    
      self.lastplayernum = 0
      y = [[0] for i in range(4)]
      # for i in range(24):
      #   self.x_feature_vector[i][37] =[1]

    
    if sute != '':
      sute = [int(s) for s in sute]
      tsumogiri = [int(s) for s in tsumogiri]
      red = [int(s) for s in red]
      
      self.x_feature_vector[self.round[playernum]+playernum*24][(sute[-1] // 9) + 2] = [1]
      self.x_feature_vector[self.round[playernum]+playernum*24][(sute[-1] % 9) + 6] = [1]
      self.x_feature_vector[self.round[playernum]+playernum*24][tsumogiri[-1] + 19] = [1]
      self.x_feature_vector[self.round[playernum]+playernum*24][18] = [red[-1]]
      for j in range(len(dora)):
        if sute[-1] == dora[j] :
          self.x_feature_vector[self.round[playernum]+playernum*24][15] = [1]
        elif (sute[-1] == dora[j]+1 or sute[-1] == dora[j]-1) :
          self.x_feature_vector[self.round[playernum]+playernum*24][16] = [1]
        if sute[-1]//9 == dora[j]//9 :
          self.x_feature_vector[self.round[playernum]+playernum*24][17] = [1]
      if sute[-1] == bakaze:
        self.x_feature_vector[self.round[playernum]+playernum*24][1] = [1]
      if sute[-1] == jikaze:
        self.x_feature_vector[self.round[playernum]+playernum*24][0] = [1]

    
    


    naki_round = line[12].split(':')[1].split('],[')
    naki = [s.split(',') for s in  line[4].split(':')[1:]]
    yaochu = [0, 8, 9, 17, 18, 26, 27, 28, 29, 30, 31, 32, 33]
    
    while naki_round[-1] == '-1':
      naki_round.pop(-1)
      if naki_round == []:
        break
    
    

    if naki_round != []: 
      for i in range(len(naki_round)):
        if naki_round[i].find(',') != -1:
          xmk_round[i]=int(naki_round[i].split(',')[1])
      max_xmk = max(xmk_round)#連カンは考えてない
      new_xmk = xmk_round.index(max_xmk)
      if  max_xmk != -1 and max_xmk==len(sute)-1:   
        xmk_count += 1
        naki_round[new_xmk] = [int(s) for s in naki_round[new_xmk].split(',')]
        if len(sute) == naki_round[new_xmk][1]+1:
          



          self.x_feature_vector[self.round[playernum]+playernum*24][26] = [1]
          self.x_feature_vector[self.round[playernum]+playernum*24][21] = [1]
          self.x_feature_vector[self.round[playernum]+playernum*24][20] = [0]
          self.x_feature_vector[self.round[playernum]+playernum*24][19] = [0]

          self.x_feature_vector[self.round[playernum]+playernum*24][(int(naki[new_xmk][1])//9)+27] = [1]
         
          if int(naki[new_xmk][1]) in dora:
            self.x_feature_vector[self.round[playernum]+playernum*24][31] = [1]
            
          if int(naki[new_xmk][1]) in yaochu:
            self.x_feature_vector[self.round[playernum]+playernum*24][32] = [1]
            
          if int(naki[new_xmk][1]) == jikaze:
            self.x_feature_vector[self.round[playernum]+playernum*24][33] = [1]
            
          if int(naki[new_xmk][1]) == bakaze:
            self.x_feature_vector[self.round[playernum]+playernum*24][34] = [1]
            
          if int(naki[new_xmk][1]) >= 31:
            self.x_feature_vector[self.round[playernum]+playernum*24][35] = [1]
        
      else:
        if naki_round[-1].find(',') == -1:
          naki_round[-1] = int(naki_round[-1])
          if len(sute) == naki_round[-1]+1: 
                                                  
            self.x_feature_vector[self.round[playernum]+playernum*24][21] = [1]
            self.x_feature_vector[self.round[playernum]+playernum*24][20] = [0]
            self.x_feature_vector[self.round[playernum]+playernum*24][19] = [0]
            self.x_feature_vector[self.round[playernum]+playernum*24][(int(naki[-1][0]))+21] = [1]
            self.x_feature_vector[self.round[playernum]+playernum*24][(int(naki[-1][1])//9)+27] = [1]
            if int(naki[-1][0]) == 1:
              if int(naki[-1][1]) in dora or (int(naki[-1][2])) in dora or (int(naki[-1][3])) in dora:
                self.x_feature_vector[self.round[playernum]+playernum*24][31] = [1]
              if int(naki[-1][1]) in yaochu or (int(naki[-1][2])) in yaochu or (int(naki[-1][3])) in yaochu:
                self.x_feature_vector[self.round[playernum]+playernum*24][32] = [1]
            else:
              if int(naki[-1][1]) in dora:
                self.x_feature_vector[self.round[playernum]+playernum*24][31] = [1]
              if int(naki[-1][1]) in yaochu:
                self.x_feature_vector[self.round[playernum]+playernum*24][32] = [1]
              if int(naki[-1][1]) == jikaze:
                self.x_feature_vector[self.round[playernum]+playernum*24][33] = [1]
              if int(naki[-1][1]) == bakaze:
                self.x_feature_vector[self.round[playernum]+playernum*24][34] = [1]
              if int(naki[-1][1]) >= 31:
                self.x_feature_vector[self.round[playernum]+playernum*24][35] = [1]



    xiangting = int(line[2].split(':')[1])
    
    if xiangting == 0:
      self.y[playernum] = [1]
    else:
      self.y[playernum] = [0]

    self.round[playernum]+=1
    
    self.lastplayernum = playernum 
    if playernum == 0:
      self.x_feature_vector1 =self.x_feature_vector[0:24]
    #   self.x_feature_vector2 =self.x_feature_vector[24:48]
    #   self.x_feature_vector3 =self.x_feature_vector[48:72]
    #   self.x_feature_vector4 =self.x_feature_vector[72:96]
      y1 = self.y[0].copy()
    #   y2 = y[1].copy()
    #   y3 = y[2].copy()
    #   y4 = y[3].copy()
    elif playernum == 1:
      self.x_feature_vector1 =self.x_feature_vector[24:48]
    #   self.x_feature_vector2 =self.x_feature_vector[48:72]
    #   self.x_feature_vector3 =self.x_feature_vector[72:96]  
    #   self.x_feature_vector4 =self.x_feature_vector[0:24]
      y1 = self.y[1].copy()
    #   y2 = y[2].copy()
    #   y3 = y[3].copy()
    #   y4 = y[0].copy()
    elif playernum == 2:
      self.x_feature_vector1 =self.x_feature_vector[48:72]
    #   self.x_feature_vector2 =self.x_feature_vector[72:96] 
    #   self.x_feature_vector3 =self.x_feature_vector[0:24]
    #   self.x_feature_vector4 =self.x_feature_vector[24:48]
      y1 = self.y[2].copy()
    #   y2 = y[3].copy()
    #   y3 = y[0].copy()
    #   y4 = y[1].copy()
    elif playernum == 3:
      self.x_feature_vector1 =self.x_feature_vector[72:96]
    #   self.x_feature_vector2 =self.x_feature_vector[0:24]
    #   self.x_feature_vector3 =self.x_feature_vector[24:48]
    #   self.x_feature_vector4 =self.x_feature_vector[48:72]
      y1 = self.y[3].copy()
    #   y2 = y[0].copy()
    #   y3 = y[1].copy()
    #   y4 = y[2].copy()

    # self.x_feature_vector_all= self.x_feature_vector1 + self.x_feature_vector2 + self.x_feature_vector3 + self.x_feature_vector4
    
    xfv1=[[[0] for j in range(36)] for i in range(24)]
    for j in range(24):
      for k in range(36):
        xfv1[j][k]=self.x_feature_vector1[j][k].copy()
    

    sutelen = len(sute)
    return xfv1 ,y1 ,lichi ,playernum ,sutelen 

  def read_data(self,fin,batch_size):
    x1_list, y1_list,y2_list,y3_list,y4_list,sutelen_list,lichi_list,pn_list = [],[],[],[],[],[],[],[]
    eofeof = False
    pre_x1 = 0
    pre_y1 = [0]*24
    
    
    for i in range(batch_size):
      line = fin.readline().replace('\n', '')
      # count+=1
      if line:
        pass
      else:
        eofeof = True
        break
      # x1,y1,y2,y3,y4,lichi,pn,sutelen = make_feature_vector(line)
      x1,y1,lichi,pn,sutelen = self.make_feature_vector_forone(line)
      
      if lichi == 1 :              
        continue

      if pn != 0:
        continue

     

      
    
      

      if pn == 0 and sutelen == 1 and pre_x1 != 0:
        
        a=[[0]]*36
        b=0
        # print(b)
        # print(pre_y1[-1])
        # input()
        while pre_x1[-1]==a:          
          pre_x1.pop(-1)
        while pre_y1[-1]==b:          
          pre_y1.pop(-1) 
        # if len(pre_x1)==len(pre_y1):
        #   print('yes3')
        array_x1 = np.array([pre_x1])
        array_y1 = np.array([pre_y1])

        x1_list.append(array_x1)
        y1_list.append(array_y1)
        pre_y1 = [0]*24
      if pn == 0:
        pre_x1 = x1
        
        pre_y1[sutelen-1] = y1
        

    # return x1_list, y1_list,y2_list,y3_list,y4_list, eofeof,sutelen_list,lichi_list,pn_list
    return x1_list, y1_list, eofeof#,sutelen_list,lichi_list,pn_list

# **Main**

In [ ]:
if __name__ == '__main__':
  with codecs.open(train_path, 'r', 'utf-8', 'ignore') as fin:
    eofeof = False
    while not eofeof:
      x_train = []
      y_train = []
      train_dataset = Dataset()
      x_train ,y_train ,eofeof = train_dataset.read_data(fin,500010)
      
      train = Train()
      with codecs.open(test_path, 'r', 'utf-8', 'ignore') as fin2:
        eofeof2 = False
        while not eofeof2:
          x_test = []
          y_test = []
          test_dataset = Dataset()
          x_test ,y_test ,eofeof2 = test_dataset.read_data(fin2,101100)
          print(len(x_test))
          break
      train.train(x_train,y_train,x_test,y_test)